In [32]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm = tqdm.pandas()

In [33]:
def setting(pat_class, payer):
    if pd.isna(pat_class):
        if 'Outpatient' in payer or 'OP' in payer:
            return 'outpatient'
        elif 'Inpatient' in payer or 'IP' in payer:
            return 'inpatient'
        else:
            return pd.NA
    else:
        return pat_class

def payer_cat(x):
    if 'Gross Charges' in x:
        return 'gross'
    elif 'Max Reimbursement' in x:
        return'max'
    elif 'Minimal Reimbursement' in x:
        return'min'
    elif 'Cash Price' in x:
        return 'cash'
    else:
        return 'payer'
    

In [34]:
file = '823970937_UTHealth-Specialty-Hospital_standardcharges.csv'
df = pd.read_csv(file)

C:\Users\adria\AppData\Local\Temp\ipykernel_33852\2498153818.py:2: DtypeWarning: Columns (0,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [35]:
df.rename(columns={
    'Procedure': 'local_code',
    'Code Type': 'line_type',
    'Revenue Code': 'rev_desc',
    'Procedure Description': 'description',
    'Payer': 'payer',
    'Plan': 'plan_name',
    'Quantity': 'drug_quantity',
}, inplace=True)

In [36]:
df['code_orig'] = df['local_code'].where(df['local_code'].str.startswith('MS'), other=pd.NA)
df['code_prefix'] = np.where(df['line_type'] == 'DRG', 'ms-drg', pd.NA)
df['code'] = df['code_orig'].str.replace('MS', '')
df['code'].fillna('none', inplace=True)
df['code_prefix'].fillna('none', inplace=True)

df.loc[df['code_prefix'] == 'ms-drg', 'ms_drg'] = df['code']

In [37]:
df['rev_code'] = df['rev_desc'].str.split(" - ").str[0]

In [38]:
cols = df.columns.tolist()
cols = cols[-4:] + cols[:-4]
df = df[cols]

In [39]:
id_vars = cols[:11]

# Define columns to be melted
value_vars = ['Inpatient Expected Reimbursement', 'Outpatient Expected Reimbursement']

# Melt the dataframe 
# Var name as setting because the var names don't matter here as these are the standard_charges for `payer`
df_melted = pd.melt(df, id_vars=id_vars, value_vars=value_vars,
                    var_name='setting', value_name='standard_charge')

# Extract the patient class from the variable column
df_melted['setting'] = df_melted['setting'].str.split().str[0].str.lower()

In [40]:
id_vars = cols[:11]
id_vars.remove('payer')
id_vars.remove('plan_name')
# Define columns to be melted
value_vars = cols[11:]
value_vars.remove('Outpatient Expected Reimbursement')
value_vars.remove('Inpatient Expected Reimbursement')

# Melt the dataframe 
# Var name as setting because the var names don't matter here as these are the standard_charges for `payer`
df_1 = df.drop(['payer', 'plan_name'], axis=1)
df_melted_2 = pd.melt(df_1, id_vars=id_vars, value_vars=value_vars,
                    var_name='payer', value_name='standard_charge')

# Extract the patient class from the variable column
# df_melted['setting'] = df_melted['setting'].str.split().str[0].str.lower()

In [41]:
df = pd.concat([df_melted, df_melted_2])

In [42]:
df['setting'] = df[['setting', 'payer']].progress_apply(lambda x: setting(pat_class=x[0], payer=x[1]), axis=1)

100%|██████████| 1068560/1068560 [00:15<00:00, 68217.08it/s]


In [43]:
df['payer_category'] = df['payer'].apply(payer_cat)

In [44]:
df['local_code'].fillna('na', inplace=True)
df['payer'] = df['payer']
df['plan_orig'] = df['plan_name']
df['plan_orig'].fillna('na', inplace=True)

In [45]:
# df['hospital_ein'] = '82-3970937'
# df['hospital_ccn'] = '452051'
# df['filename'] = file
# df['url'] = 'https://coc.ardenthealthservices.com/2023/easttexas/823970937_UTHealth-Specialty-Hospital_standardcharges.csv'
# df['file_last_updated'] = '2023-01-10'

In [46]:
df.dropna(subset=['standard_charge'], inplace=True)
df['rev_code'].fillna('na', inplace=True)
df['setting'].fillna('na', inplace=True)

In [47]:
df.to_csv('specialty.csv', index=False)